In [1]:
from fenics import *
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

# Heat Equation with time dependence

$\begin{aligned}
\frac{\partial u}{\partial t} &=\nabla^{2} u+f & & \text { in } \Omega \times(0, T] \\
u &=u_{\mathrm{D}} & & \text { on } \partial \Omega \times(0, T] \\
u &=u_{0} & & \text { at } t=0
\end{aligned}$

where, $ f(x, y, t)=\beta-2-2 \alpha$

## Set parameters


+ Time = 2
+ Steps = 10
+ $\alpha = 3$
+ $\beta = 1.2$

In [3]:
T = 20            # final time
num_steps = 50     # number of time steps
dt = T / num_steps # time step size
alpha = 3          # parameter alpha
beta = 1.2         # parameter beta

## Create mesh and boundary condition

$u_{\mathrm{D}}(x, y, t)=1+x^{2}+\alpha y^{2}+\beta t$

In [4]:
# Create mesh and define function space
nx = ny = 80
mesh = UnitSquareMesh(nx, ny)
V = FunctionSpace(mesh, 'P', 1)

# Define boundary condition
u_D = Expression('1 + x[0]*x[0] + alpha*x[1]*x[1] + beta*t', degree=2, alpha=alpha, beta=beta, t=0)

def boundary(x, on_boundary):
    return on_boundary

bc = DirichletBC(V, u_D, boundary)

## Initial value and variational form

Discretization

$$\left(\frac{\partial u}{\partial t}\right)^{n+1}=\nabla^{2} u^{n+1}+f^{n+1}$$


$$\frac{u^{n+1}-u^{n}}{\Delta t}=\nabla^{2} u^{n+1}+f^{n+1}$$

Using the initial condition $u_0$:

$$\begin{aligned}
u^{0} &=u_{0} \\
u^{n+1}-\Delta t \nabla^{2} u^{n+1} &=u^{n}+\Delta t f^{n+1}, \quad n=0,1,2, \ldots
\end{aligned}$$

Now, all terms to one side of the equation:


$$u^{n+1}-\Delta t \nabla^{2} u^{n+1}-u^{n}-\Delta t f^{n+1}=0, \quad n=0,1,2, \ldots$$

Writting in the standard notation:

$$a(u, v)=L_{n+1}(v)$$

where

$$\begin{aligned}
a(u, v) &=\int_{\Omega}(u v+\Delta t \nabla u \cdot \nabla v) \mathrm{d} x \\
L_{n+1}(v) &=\int_{\Omega}\left(u^{n}+\Delta t f^{n+1}\right) v \mathrm{d} x
\end{aligned}$$

Finally, using the abstract formulation:

$$F_{n+1}(u ; v)=0$$

where

$$F_{n+1}(u ; v)=\int_{\Omega}\left(u v+\Delta t \nabla u \cdot \nabla v-\left(u^{n}+\Delta t f^{n+1}\right) v\right) \mathrm{d} x$$

We need approximate the initial condition into a variational problem:

$$a_{0}(u, v)=L_{0}(v)$$

with

$$\begin{aligned}
a_{0}(u, v) &=\int_{\Omega} u v \mathrm{d} x \\
L_{0}(v) &=\int_{\Omega} u_{0} v \mathrm{d} x
\end{aligned}$$

In [5]:
# Define initial value
u_n = interpolate(u_D, V)
#u_n = project(u_D, V)

# Define variational problem
u = TrialFunction(V)
v = TestFunction(V)
f = Constant(beta - 2 - 2*alpha)

F = u*v*dx + dt*dot(grad(u), grad(v))*dx - (u_n + dt*f)*v*dx #Abstract formulation
a, L = lhs(F), rhs(F)

## Iterate at the time

In [6]:
# Time-stepping
u = Function(V)
t = 0

for n in range(num_steps):
    # Update current time
    t = round(t + dt, 2)
    u_D.t = t #Update time parameter at the boundary

    # Compute solution
    solve(a == L, u, bc)  
    
    # Compute error at vertices
    u_e = interpolate(u_D, V)
    error = np.abs(u_e.vector() - u.vector()).max()
    print(' time = %.2f \n error = %.3g' % (t, error))

    # Update previous solution
    u_n.assign(u)
    #plot(u)
    #plt.show()

 time = 0.40 
 error = 3.57e-13
 time = 0.80 
 error = 4.73e-13
 time = 1.20 
 error = 5.58e-13
 time = 1.60 
 error = 6.38e-13
 time = 2.00 
 error = 7.19e-13
 time = 2.40 
 error = 8.06e-13
 time = 2.80 
 error = 8.85e-13
 time = 3.20 
 error = 9.64e-13
 time = 3.60 
 error = 1.05e-12
 time = 4.00 
 error = 1.13e-12
 time = 4.40 
 error = 1.22e-12
 time = 4.80 
 error = 1.3e-12
 time = 5.20 
 error = 1.38e-12
 time = 5.60 
 error = 1.46e-12
 time = 6.00 
 error = 1.55e-12
 time = 6.40 
 error = 1.63e-12
 time = 6.80 
 error = 1.71e-12
 time = 7.20 
 error = 1.79e-12
 time = 7.60 
 error = 1.87e-12
 time = 8.00 
 error = 1.96e-12
 time = 8.40 
 error = 2.04e-12
 time = 8.80 
 error = 2.12e-12
 time = 9.20 
 error = 2.21e-12
 time = 9.60 
 error = 2.29e-12
 time = 10.00 
 error = 2.37e-12
 time = 10.40 
 error = 2.46e-12
 time = 10.80 
 error = 2.54e-12
 time = 11.20 
 error = 2.62e-12
 time = 11.60 
 error = 2.71e-12
 time = 12.00 
 error = 2.78e-12
 time = 12.40 
 error = 2.86e-12
 t

In [8]:
@interact
def show_solution_time(t = (dt,T,4*dt)):
    u_D.t = t
    solve(a == L, u, bc)
    u_n.assign(u)
    return plot(u)

interactive(children=(FloatSlider(value=10.000000000000002, description='t', max=20.0, min=0.4, step=1.6), Out…